In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[11]
# print the array

In [3]:
Data_train = load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
Data_test =  load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)


Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]



for idx in range(len(Data_train)//39):
    if(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    Xtrain_data.append(Data_train[39*idx:39*(idx+1)][:,:].tolist())
    Ytrain_data.append(Data_train[39*idx:39*(idx+1)][:,-1].tolist())
for idx in range(len(Data_test)//39):
    if(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    Xtest_data.append(Data_test[39*idx:39*(idx+1)][:,:].tolist())
    Ytest_data.append(Data_test[39*idx:39*(idx+1)][:,-1].tolist())

Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)
Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)
Ytrain_data=torch.cat(((Ytrain_data[1:]-Ytrain_data[:-1]),torch.tensor(0).unsqueeze(-1)))
Ytrain_data[::39]=0

Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)
Ytest_data=torch.cat(((Ytest_data[1:]-Ytest_data[:-1]),torch.tensor(0).unsqueeze(-1)))
Ytest_data[::39]=0


Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T




Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [4]:
Xtrain_data.size()

torch.Size([74100, 7])

In [5]:
Ytrain_data

tensor([1, 0, 2,  ..., 1, 1, 1])

In [6]:
Xtrain_data

tensor([[ 2.5800e+04,  3.8780e-01,  1.7981e-01,  ...,  2.9070e-02,
         -9.6899e-03,  2.6250e+04],
        [ 2.5800e+04,  6.1143e-01,  2.2340e-01,  ...,  2.9070e-02,
         -9.6899e-03,  2.6175e+04],
        [ 2.5800e+04,  7.1929e-01,  2.2576e-01,  ...,  2.9070e-02,
         -9.6899e-03,  2.5925e+04],
        ...,
        [ 2.8250e+03,  2.8344e-01,  1.0433e-01,  ...,  3.8938e-02,
         -3.5398e-03,  2.8800e+03],
        [ 2.8250e+03,  2.7932e-01,  1.0208e-01,  ...,  3.8938e-02,
         -3.5398e-03,  2.8800e+03],
        [ 2.8250e+03,  2.5743e-01,  1.1438e-01,  ...,  3.8938e-02,
         -3.5398e-03,  2.8800e+03]])

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        
#         print(data.size(0),bsz,bptt)
        
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


In [8]:


bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
#     if(seq_len!=bptt*bs):
#         print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target


In [9]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE=Xtrain_data.shape[1]
EMB_SIZE=64
HIDDEN_SIZE=64
NUM_LAYERS=6
PROJ_SIZE=3
BATCH_SIZE=16

In [10]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers,proj_size, dropout=0.5, tie_weights=False, attention=True,
                 attention_width=3, cuda=True):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
#         self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder = nn.Linear(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError("""An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        if attention:
            self.decoder = nn.Linear(nhid, ntoken)
        else:
            self.decoder = nn.Linear(nhid, ntoken)
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.softmax = nn.Softmax()
        if attention:
            self.AttentionLayer = AttentionLayer(cuda,nhid)
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
        self.attention = attention
        self.attention_width = attention_width
        self.generator = nn.Linear(nhid, proj_size)
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        #print("input size:",input.size())
#         emb = self.drop(self.encoder(input))
        emb=self.encoder(input)
        #print("emb size:",emb.size())
        output, _ = self.rnn(emb)
        #print("rnn output",output.size())
        if self.attention:
            output = self.AttentionLayer.forward(output, self.attention_width)
        return self.generator(output)
        #
#         output = self.drop(output)
#         decoded = self.decoder(output.view(output.size(0) * output.size(1), output.size(2)))
#         return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())


class AttentionLayer(nn.Module):
    """Implements an Attention Layer"""

    def __init__(self, cuda, nhid):
        super(AttentionLayer, self).__init__()
        self.nhid = nhid
        self.weight_W = nn.Parameter(torch.Tensor(nhid,nhid))
        self.weight_proj = nn.Parameter(torch.Tensor(nhid, 1))
        self.softmax = nn.Softmax()
        self.weight_W.data.uniform_(-0.1, 0.1)
        self.weight_proj.data.uniform_(-0.1,0.1)
        self.cuda = cuda

    def forward(self, inputs, attention_width=3):
        results = None
        for i in range(inputs.size(0)):
            if(i<attention_width):
                output = inputs[i]
                output = output.unsqueeze(0)

            else:
                lb = i - attention_width
                if(lb<0):
                    lb = 0
                selector = torch.from_numpy(np.array(np.arange(lb, i)))
                if self.cuda:
                    selector = Variable(selector).cuda()
                else:
                    selector = Variable(selector)
                vec = torch.index_select(inputs, 0, selector)
                u = batch_matmul(vec, self.weight_W, nonlinearity='tanh')
                a = batch_matmul(u, self.weight_proj)
                a = self.softmax(a)
                output = None
                for i in range(vec.size(0)):
                    h_i = vec[i]
                    a_i = a[i].unsqueeze(1).expand_as(h_i)
                    h_i = a_i * h_i
                    h_i = h_i.unsqueeze(0)
                    if(output is None):
                        output = h_i
                    else:
                        output = torch.cat((output,h_i),0)
                output = torch.sum(output,0)
                output = output.unsqueeze(0)

            if(results is None):
                results = output

            else:
                results = torch.cat((results,output),0)

        return results

In [11]:
torch.manual_seed(0)



#lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)

#lstm = RNNmodel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=INPUT_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
lstm = RNNModel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=EMB_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,proj_size=PROJ_SIZE,
                attention_width=38)
# for p in lstm.parameters():
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)

lstm = lstm.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)

In [12]:
(Ytest_data==2).all()

tensor(False)

In [13]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    
    
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
    i=0
    while(i<Xtest_data.size(0)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        _,predicted=torch.max(logits,-1)
        correct += (tgt.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt.squeeze()).sum().item()
        tot1+=(1== tgt.squeeze()).sum().item()
        tot2+=(2== tgt.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
        
        conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
        conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
        conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
        conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
        conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
        conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
        conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

        
        
        
        i+=targets.size()[0]

    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
        
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc) 
    return losses / Xtrain_data.size(0),[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    with torch.no_grad():
        
        i=0
        while(i<Xtest_data.size(0)):
            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)
#             print(src.shape)
            if(src.shape[1]!=BATCH_SIZE):
                
                break
#             print(tgt.shape)
            src=src[:-1]
            tgt=tgt[1:]


            logits = model(src)
#            print(logits.reshape(-1, logits.shape[-1]).shape,'AA')
#            print(tgt.reshape(-1).shape,'BB')
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
            conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
            conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
            conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
            conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
            conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
            conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
            conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

            
            i+=targets.size()[0]
            
#     print(total,tot0,tot1,tot2)
#     print(correct)
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
    
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    if(prec+reca==0):
        f1sc=0
    else:
        f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc)
    return losses / Xtest_data.size(0),correct/total,prec,reca,f1sc,[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [14]:
evaluate(lstm)

7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227


(0.0016733451072986308,
 0.4414747807017544,
 0.14715826023391812,
 0.3333333333333333,
 0.20417736363348227,
 [3221, 1344, 2731, 0, 0, 0, 0, 0, 0])

In [15]:
Data_test

array([[43800.0, 0.5857238535412305, 0.5237050024703537, ...,
        0.0045662100456621, -0.02054794520547945, 43225.0],
       [43800.0, 0.6990164914503354, 0.5673475730048468, ...,
        0.005707762557077625, -0.02054794520547945, 43775.0],
       [43800.0, 0.7667760430820563, 0.5981295717365939, ...,
        0.012557077625570776, -0.02054794520547945, 43875.0],
       ...,
       [3800.0, 1.5087932452654602, 0.6648274736208902, ...,
        0.09868421052631579, -0.06447368421052632, 3632.5],
       [3800.0, 1.4368026784935448, 0.6070113919629829, ...,
        0.09868421052631579, -0.06447368421052632, 3625.0],
       [3800.0, 1.5626597950648295, 0.7354417916835297, ...,
        0.09868421052631579, -0.06447368421052632, 3622.5]], dtype=object)

In [16]:
Data_test[:,-1]

array([43225.0, 43775.0, 43875.0, ..., 3632.5, 3625.0, 3622.5],
      dtype=object)

In [17]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss,_ = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss,acc,prec,reca,f1sc,confusion = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_confusion=confusion
        best_acc=acc
        best_prec=prec
        best_reca=reca
        best_f1sc=f1sc
        best_model = lstm



7904
3122
Acc: 0.3949898785425101
Prec 0.26318539463978363
Recall 0.3248412675095403
F1 0.2907809549733498
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 1, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.338s
7904
3369
Acc: 0.4262398785425101
Prec 0.14211592002024803
Recall 0.33323442136498516
F1 0.19925479063165363
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 2, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.326s
7904
3370
Acc: 0.4263663967611336
Prec 0.1421221322537112
Recall 0.3333333333333333
F1 0.19927857607474422
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 3, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.313s
7904
3370
Acc: 0.4263663967611336
Prec 0.1421221322537112
Recall 0.3333333333333333
F1 0.19927857607474422
7296
3221
Acc: 0.4414747807017544
Prec 0.1471582

7904
3363
Acc: 0.4254807692307692
Prec 0.20082936179416613
Recall 0.332674246653152
F1 0.25046037396129395
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 31, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.321s
7904
3370
Acc: 0.4263663967611336
Prec 0.1421221322537112
Recall 0.3333333333333333
F1 0.19927857607474422
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 32, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.319s
7904
3370
Acc: 0.4263663967611336
Prec 0.1421221322537112
Recall 0.3333333333333333
F1 0.19927857607474422
7296
3221
Acc: 0.4414747807017544
Prec 0.14715826023391812
Recall 0.3333333333333333
F1 0.20417736363348227
Epoch: 33, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.315s
7904
3370
Acc: 0.4263663967611336
Prec 0.2088365615900747
Recall 0.33334443236608524
F1 0.25679433936836743
7296
3221
Acc: 0.4414747807017544
Prec 0.14715

7904
3355
Acc: 0.4244686234817814
Prec 0.2799105753931216
Recall 0.33632256400714255
F1 0.3055345011160848
7296
3064
Acc: 0.4199561403508772
Prec 0.251046772875817
Recall 0.3216720583631774
F1 0.28200480854347315
Epoch: 61, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.320s
7904
3385
Acc: 0.42826417004048584
Prec 0.2882504526118478
Recall 0.33834650527367277
F1 0.311295904192984
7296
3190
Acc: 0.43722587719298245
Prec 0.28662109375
Recall 0.33909350759557166
F1 0.3106571328895345
Epoch: 62, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.318s
7904
3375
Acc: 0.426998987854251
Prec 0.2835733879399465
Recall 0.33832300143960986
F1 0.30853821110605534
7296
3209
Acc: 0.43983004385964913
Prec 0.2883220996732026
Recall 0.3381260450828967
F1 0.31124431316004003
Epoch: 63, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.345s
7904
3399
Acc: 0.43003542510121456
Prec 0.2909521756145929
Recall 0.34061919545070235
F1 0.3138327685921286
7296
3109
Acc: 0.42612390350877194
Prec 0.25527447089

7904
3438
Acc: 0.4349696356275304
Prec 0.29374081934381896
Recall 0.34699624265097134
F1 0.318155345371146
7296
3112
Acc: 0.42653508771929827
Prec 0.2739375320020481
Recall 0.3316899240774405
F1 0.30006010553060225
Epoch: 91, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.331s
7904
3394
Acc: 0.42940283400809715
Prec 0.2853421628985647
Recall 0.34186718375101605
F1 0.3110576146756826
7296
3164
Acc: 0.4336622807017544
Prec 0.2787037037037037
Recall 0.33450890771470165
F1 0.3040670389551272
Epoch: 92, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.304s
7904
3388
Acc: 0.42864372469635625
Prec 0.2850861420849617
Recall 0.34053007674654706
F1 0.3103513077870341
7296
3159
Acc: 0.4329769736842105
Prec 0.28070746527777773
Recall 0.3355511722719507
F1 0.30568892117749996
Epoch: 93, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.348s
7904
3442
Acc: 0.4354757085020243
Prec 0.2945383742319283
Recall 0.3465816611334724
F1 0.3184476958380299
7296
3114
Acc: 0.42680921052631576
Prec 0.27443

7904
3409
Acc: 0.4313006072874494
Prec 0.2844070061615049
Recall 0.3449935201235257
F1 0.3117842140443867
7296
3108
Acc: 0.42598684210526316
Prec 0.27160138248847926
Recall 0.33107172724097583
F1 0.2984023589898125
Epoch: 120, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.315s
7904
3441
Acc: 0.4353491902834008
Prec 0.2896196034969811
Recall 0.34819200020892294
F1 0.3162163512090257
7296
3155
Acc: 0.43242872807017546
Prec 0.2786757662835249
Recall 0.33903647766797085
F1 0.3059070016413917
Epoch: 121, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.320s
7904
3402
Acc: 0.43041497975708504
Prec 0.28498225857882425
Recall 0.3444343247381118
F1 0.3119004945140222
7296
3123
Acc: 0.42804276315789475
Prec 0.2743055555555556
Recall 0.33410947085965853
F1 0.3012683120616614
Epoch: 122, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.318s
7904
3425
Acc: 0.4333248987854251
Prec 0.28707893036025006
Recall 0.347197657451205
F1 0.3142891730202258
7296
3108
Acc: 0.42598684210526316
Prec 0.27

...

7904
3487
Acc: 0.441169028340081
Prec 0.292913978819096
Recall 0.35523923312212524
F1 0.3210800634461175
7296
3083
Acc: 0.4225603070175439
Prec 0.2731539936401048
Recall 0.33392492883479746
F1 0.3004977590569655
Epoch: 882, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.330s
7904
3470
Acc: 0.4390182186234818
Prec 0.2909119729256031
Recall 0.3534800364309428
F1 0.3191584417399016
7296
3088
Acc: 0.4232456140350877
Prec 0.27384960718294055
Recall 0.33451663801718046
F1 0.30115822710390094
Epoch: 883, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.325s
7904
3479
Acc: 0.4401568825910931
Prec 0.29232616323863264
Recall 0.354414640277084
F1 0.32039008958333764
7296
3083
Acc: 0.4225603070175439
Prec 0.2731539936401048
Recall 0.33392492883479746
F1 0.3004977590569655
Epoch: 884, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.324s
7904
3491
Acc: 0.4416751012145749
Prec 0.29337056698505254
Recall 0.3556459800282699
F1 0.3215205014016584
7296
3104
Acc: 0.42543859649122806
Prec 0.274603

7904
3488
Acc: 0.44129554655870445
Prec 0.2922770254603184
Recall 0.35562671994202383
F1 0.3208548202304887
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 911, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.337s
7904
3499
Acc: 0.44268724696356276
Prec 0.2934901691208683
Recall 0.35695893031439646
F1 0.32212800945726566
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 912, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.330s
7904
3490
Acc: 0.4415485829959514
Prec 0.2921732873226381
Recall 0.3562130100250381
F1 0.3210306157050073
7296
3088
Acc: 0.4232456140350877
Prec 0.27384960718294055
Recall 0.33451663801718046
F1 0.30115822710390094
Epoch: 913, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.310s
7904
3491
Acc: 0.4416751012145749
Prec 0.2929654464547744
Recall 0.35605664424009165
F1 0.32144450932727664
7296
3067
Acc: 0.4203673245614035
Prec 0

7904
3491
Acc: 0.4416751012145749
Prec 0.29272554115279487
Recall 0.3564562094191615
F1 0.321462631096967
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 940, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.344s
7904
3484
Acc: 0.4407894736842105
Prec 0.29148759079903147
Recall 0.355275468199639
F1 0.32023594685769946
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 941, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.311s
7904
3501
Acc: 0.4429402834008097
Prec 0.29325139732946787
Recall 0.3577117058886898
F1 0.32229002557710523
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 942, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.335s
7904
3494
Acc: 0.44205465587044535
Prec 0.29297902352930816
Recall 0.3566308559639347
F1 0.3216864867333711
7296
3088
Acc: 0.4232456140350877
Prec 0.27

7904
3494
Acc: 0.44205465587044535
Prec 0.2926752635353455
Recall 0.3567085491931983
F1 0.3215348661118271
7296
3088
Acc: 0.4232456140350877
Prec 0.27384960718294055
Recall 0.33451663801718046
F1 0.30115822710390094
Epoch: 969, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.331s
7904
3509
Acc: 0.44395242914979755
Prec 0.29484218158213654
Recall 0.3580479412926456
F1 0.32338561244236563
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 970, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.344s
7904
3485
Acc: 0.440915991902834
Prec 0.2914827604326682
Recall 0.3558516385763206
F1 0.3204668810293635
7296
3088
Acc: 0.4232456140350877
Prec 0.27384960718294055
Recall 0.33451663801718046
F1 0.30115822710390094
Epoch: 971, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.321s
7904
3494
Acc: 0.44205465587044535
Prec 0.29340464455008775
Recall 0.3562423898176168
F1 0.32178449597585895
7296
3067
Acc: 0.4203673245614035
Prec 0

7904
3502
Acc: 0.4430668016194332
Prec 0.2936280100148125
Recall 0.35747764687447975
F1 0.3224221720880507
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 998, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.350s
7904
3488
Acc: 0.44129554655870445
Prec 0.29206243489394396
Recall 0.3561372754486131
F1 0.3209329413894623
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 999, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.317s
7904
3485
Acc: 0.440915991902834
Prec 0.2919471997393927
Recall 0.3557184501832973
F1 0.3206932633807172
7296
3067
Acc: 0.4203673245614035
Prec 0.2725694444444445
Recall 0.33355031231369353
F1 0.29999227811969426
Epoch: 1000, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.313s


In [18]:


from datetime import datetime
now = datetime.now()
now.strftime("%m/%d/%Y, %H:%M:%S")

date_time = now.strftime("%m_%d_%Y")
PATH='best_model_seq_attLSTM_'+date_time+str(MBR_NO)+'_'+str(BRN_NO)

In [19]:
torch.save(best_model.state_dict(), PATH)
file_name='result_attLSTM_norm'+date_time+'Mid.txt'
text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
print(text_to_append)
with open(file_name, "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    # If file is not empty then append '\n'
    data = file_object.read(100)
    if len(data) > 0:
        file_object.write("\n")
    # Append text at the end of file
    file_object.write(text_to_append)

best_model_seq_attLSTM_12_04_20212_155	Acc:0.44997258771929827	prec:0.2970433501683502	recall:0.34758520697487	f1sc:0.32033292104319766
